In [ ]:
!pip install langchain langchain-community openai tiktoken faiss-cpu langchain-openai langchainhub python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os

import glob
from langchain import hub
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

import openai as OpenAI
from google.colab import userdata
import tiktoken
import openai
from langchain.docstore.document import Document

from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

from docx import Document as format_doc
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
Openai_API_key=userdata.get('Openai_API_key')
Pinecone_API_key=userdata.get('Pinecone_API_key')

##Utils

In [ ]:
# # Read text from a file
# def read_text_file(file_path):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         text = file.read()
#     return text

def read_text_files(directory_path):
    texts = {}
    # List all files in the given directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Check if the file is a text file
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                texts[filename] = file.read()
    return texts

In [ ]:
import openai

def generate_questions(chunk, num_questions=1, API_KEY='your-api-key'):
    questions = []
    openai.api_key = API_KEY  # Set the API key

    # Assemble the prompt for the OpenAI API
    prompt = f"Given the text: \"{chunk}\" ONLY Generate {num_questions} hypothetical questions. DO NOT GENERATE MORE QUESTIONS:"

    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",  # Use the correct endpoint for completions

             messages=[
        {
            "role": "system",
            "content": prompt,
        },
    ],
)


        # Processing the output to extract questions
        text_output = response.choices[0].message.content
        questions = text_output.split('\n')[:num_questions]  # Limit the number of questions
    except Exception as e:
        print(f"An error occurred: {e}")

    return questions


In [ ]:
def save_to_docx(question, retrieved_document, relevance, explanation, filename="Relevance_Assessment.docx"):
    doc = format_doc()
    doc.add_heading('Reasoning of Context Retrieval', level=1)

    # User's question
    doc.add_heading('User\'s Question:', level=2)
    para = doc.add_paragraph()
    para.add_run(question).bold = True

    # Relevancy
    doc.add_heading('Relevancy:', level=2)
    relevancy_text = "The context is relevant to the question." if relevance.lower() == 'yes' else "The context is not relevant to the question."
    doc.add_paragraph(relevancy_text)

    # Reasoning
    doc.add_heading('Reasoning:', level=2)
    explanation_points = explanation.split('. ')  # Split by period and space to ensure that sentences are captured correctly
    for point in explanation_points:
        if point:  # Check if point is not empty
            if point[-1] != '.':  # Check if the last character is not a period
                point += '.'  # Add a period if it's missing at the end of the sentence
            doc.add_paragraph(point, style='ListBullet')
        # Retrieved context
    doc.add_heading('Retrieved Context:', level=2)
    doc.add_paragraph(retrieved_document)

    # Save the document
    doc.save(filename)
    print(f"Document saved as '{filename}'")

In [ ]:
def save_best_answer_to_docx(question, answer_1, answer_2, explanation, best_answer, filename="Best_Answer_Explanation.docx"):
    doc = format_doc()
    doc.add_heading('Best Answer Analysis', level=1)

    # User's question
    doc.add_heading('User\'s Question:', level=2)
    para = doc.add_paragraph()
    para.add_run(question).bold = True

    doc.add_heading('Answer Relevancy:', level=2)

    # Determine which answer is more accurate or if neither is correct
    if best_answer == 'Answer 1':
        best_answer_description = "First answer is more accurate and detailed."
    elif best_answer == 'Answer 2':
        best_answer_description = "Second answer is more accurate and detailed."
    else:
        best_answer_description = "Both answers are not correct."

    # Add the determined relevancy description to the document
    doc.add_paragraph(best_answer_description)

    # Explanation
    doc.add_heading('Explanation:', level=2)
    explanation_points = explanation.split('. ')  # Assuming each sentence ends with a period followed by a space
    for point in explanation_points:
        if point:  # Check if the point is not empty
            if point[-1] != '.':  # Check if the last character is not a period
                point += '.'  # Add a period if it's missing at the end of the sentence
            doc.add_paragraph(point, style='ListBullet')

    # Best Answer
    doc.add_heading('Answers:', level=2)
    doc.add_heading('Answer 1:', level=3)
    doc.add_paragraph(answer_1)

    doc.add_heading('Answer 2:', level=3)
    doc.add_paragraph(answer_2)

    # Save the document
    doc.save(filename)
    print(f"Document saved as '{filename}'")

## Data Preparation

In [ ]:
  # Read data
file_path = '/content/drive/MyDrive/vector_embedding'  # Replace with your actual text file path
text = read_text_files(file_path)

In [ ]:
# Tokenize the text using the defined tokenizer
tokenizer = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200, separators='\n')

docs=[]

for filename, content in text.items():

  tokenized_texts = tokenizer.split_text(content)  # Tokenize the content of each file

  for chunk in tokenized_texts:
    docs.append(Document(page_content=chunk, metadata={'source_document':filename}))

print(f"Number of chunks: {len(docs)}")
print(f"Source document of 2nd chunk: {docs[1].metadata['source_document']}")

Number of chunks: 19
Source document of 2nd chunk: diagnosis.txt


In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=Openai_API_key,model="text-embedding-3-large")
persisted_vectorstore = FAISS.from_documents(docs, embeddings)

# # Persist the vectors locally on disk
# vectorstore.save_local("Embeddings")

# Load from local storage
# persisted_vectorstore = FAISS.load_local("Embeddings", embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Question Generation

In [ ]:
questions = []

API_KEYYY = Openai_API_key  # Use your actual API key
# Create a separate list for the documents that contain questions
question_documents = []

for doc in docs:
  # print(doc.page_content)
  print(doc.metadata['source_document'])
  gen_questions = generate_questions(doc.page_content, API_KEY=API_KEYYY)
  # print(gen_questions)
  for quest in gen_questions:
      # Create a new document for each question and append to the separate list
      question_doc = Document(page_content=quest, metadata={'source_document': doc.metadata['source_document']})
      question_documents.append(question_doc)

# Now, if you want to combine the original docs with the new question docs
# docs.extend(question_documents)

# print(f"Total documents now: {len(docs)}")


diagnosis.txt
diagnosis.txt
diagnosis.txt
diagnosis.txt
diagnosis.txt
causes-inheritance.txt
causes-inheritance.txt
causes-inheritance.txt
causes-inheritance.txt
causes-inheritance.txt
duchenne-muscular-dystrophy.txt
duchenne-muscular-dystrophy.txt
duchenne-muscular-dystrophy.txt
duchenne-muscular-dystrophy.txt
duchenne-muscular-dystrophy.txt
signs-and-symptoms.txt
signs-and-symptoms.txt
signs-and-symptoms.txt
signs-and-symptoms.txt


In [ ]:
print(f"Number of generated questions: {len(question_documents)}")
print(f"Generated Questions:\n{chr(10).join(ques.page_content for ques in question_documents)}")

Number of generated questions: 19
Generated Questions:
What role do CK and other enzyme levels play in the diagnosis of Duchenne Muscular Dystrophy?
What information can genetic testing provide regarding muscle disorders like muscular dystrophy?
What are the implications for female relatives of men and boys with Duchenne muscular dystrophy in terms of genetic testing and potential symptoms?
How does the amount of functional dystrophin protein found in a muscle biopsy sample help distinguish between Duchenne muscular dystrophy (DMD) and Becker muscular dystrophy (BMD)?
What is the relationship between the quantity of dystrophin detected in a western blot analysis and the severity of the disease in patients with suspected DMD?
What is the role of the dystrophin protein in protecting muscles from degeneration in individuals with Becker Muscular Dystrophy?
What are the potential consequences of mutations in the gene encoding the middle section of the dystrophin protein in terms of muscle f

In [ ]:
ques_embeddings = OpenAIEmbeddings(openai_api_key=Openai_API_key,model="text-embedding-3-large")
persistedq_vectorstore = FAISS.from_documents(question_documents, ques_embeddings)

# Persist the vectors locally on disk
# ques_vectorstore.save_local("Question_Embeddings")

# Load from local storage
# persistedq_vectorstore = FAISS.load_local("Question_Embeddings", ques_embeddings)

In [ ]:
embed_result = ques_embeddings.embed_query(question_documents[0].page_content)
len(embed_result)

3072

## QA Generation

### User Input

In [ ]:
query_question = input("Please enter your question: ")

Please enter your question: What are some of the clinical implications for female carriers of the DMD gene, particularly those who show symptoms of the disease?


In [ ]:
# Relevant question retreival from hypothetical questions
retriever_FAISS=persistedq_vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

Ques_docs = retriever_FAISS.invoke(query_question)

print(f"The relevant questions are: \n{chr(10).join(Ques_docs[i].page_content for i in range(len(Ques_docs)))}")

The relevant questions are: 
What are the implications for a female who is found to be a carrier of DMD and what measures can be taken to manage any symptoms that may arise?
What are the implications for female relatives of men and boys with Duchenne muscular dystrophy in terms of genetic testing and potential symptoms?


### Relevant context retreival for each question

In [ ]:
retriever_text_FAISS=persisted_vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
contexts = []  # This will hold all the context information
sources = []
for relevant_quest in Ques_docs:
    # Ensure we have a string input for the retriever
    if hasattr(relevant_quest, 'page_content'):
        quest_text = relevant_quest.page_content
        source = relevant_quest.metadata['source_document']


        # Optional: Extract the main content after the first period
        # If you want to process text after the first period uncomment the following line
        # quest_text = quest_text.split('.', 1)[1] if '.' in quest_text else quest_text

        # Retrieve context using the FAISS retriever
        retrieved_context = retriever_text_FAISS.invoke(quest_text)

        # Append the retrieved context to the contexts list
        contexts.append(retrieved_context)
        sources.append(source)

    else:
        print("Error: Document does not have page_content")


In [ ]:
from operator import attrgetter

def extract_unique_content(contexts):
    # Set to keep track of unique page contents
    unique_contents = set()
    # List to store unique Document objects
    unique_documents = []

    # Iterate through each sublist in the main list 'contexts'
    for sublist in contexts:
        for document in sublist:
            # Check if the document's page_content is already in the set
            if document.page_content not in unique_contents:
                # If not, add the page_content to the set
                unique_contents.add(document.page_content)
                # Also add the Document to the list of unique_documents
                unique_documents.append(document)

    # Create an attribute getter for 'page_content'
    get_page_content = attrgetter('page_content')

    # Use the getter to extract 'page_content' from each Document in the list
    page_contents = [get_page_content(doc) for doc in unique_documents]

    # Join all page_content into a single string separated by newlines
    all_content = '\n'.join(page_contents)

    # Return the combined content
    return all_content

# Example usage (assuming 'contexts' is defined as per your data structure)
all_document_content = extract_unique_content(contexts)
# print(f"All Document page_content combined into one string:\n{all_document_content}")


###Answer generation

In [ ]:
# Answer generation chain

template = """Answer the question based only on the following context:
{text}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.7,  api_key=Openai_API_key)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"text": lambda x: all_document_content, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser



# document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
Answer= chain.invoke(query_question)

##Reasoning for Context Retrieval

In [ ]:

# Updated data model with explanation
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents with explanation."""
    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")
    explanation: str = Field(description="Explanation of why the document is or is not relevant to the question")

# Initialize the LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=Openai_API_key)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Updated system prompt to include reasoning
system = """You are a grader tasked with evaluating the relevance of a retrieved document segment in response to a user's question. Determine whether this segment is the most pertinent excerpt from the document, effectively and precisely addressing the user's query.
Grade the document as relevant ('yes') or not relevant ('no') and explain your reasoning based on the presence of keyword(s) or semantic meanings related to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n{Retrieved_document} \n\n User question: {question} \n\n User document: {document}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

# Assuming all_document_content contains the content of the retrieved document and docs is the full document
doc_txt = all_document_content
response = retrieval_grader.invoke({"question": query_question, "Retrieved_document": doc_txt, "document": docs})

print(f"Relevance: {response.binary_score}")
print(f"Explanation: {response.explanation}")

# Example usage
save_to_docx( question= query_question, retrieved_document= doc_txt, relevance= response.binary_score, explanation= response.explanation)


Relevance: yes
Explanation: The retrieved document segment provides information about female carriers of the DMD gene, particularly those who show symptoms of the disease. It explains that females with dystrophin deficiency may experience weaker muscles in the back, legs, and arms that fatigue easily. It also mentions that manifesting carriers may have heart problems, which can be serious and life-threatening if untreated. Additionally, it discusses rare instances where a girl may lack a second X chromosome or have sustained serious damage to it, leading to dystrophinopathy development similar to boys. The segment emphasizes the importance of diagnostic tests, carrier status management, and close cardiac monitoring for female relatives of boys with DMD who are carriers.
Document saved as 'Relevance_Assessment.docx'


/usr/local/lib/python3.10/dist-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


##Reasoning of Correct Answer

In [ ]:


# # Define a new data model for assessing and comparing two answers
# class EvaluateAnswers(BaseModel):
#     best_answer: str = Field(description="Which answer is more correct or if both are equally valid/invalid.")
#     explanation: str = Field(description="Explanation of why one answer is better or if both are equal.")

# # Initialize the language model with structured output
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=Openai_API_key)
# structured_llm_evaluator = llm.with_structured_output(EvaluateAnswers)
# # System prompt to evaluate and compare answers
# system = """Assess which of the two provided answers more accurately and comprehensively addresses the question, using detailed logical reasoning based on the given context to justify your decision."""
# evaluate_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system),
#         ("human", "Context: {context} \n\n Question: {question} \n\n Answer 1: {answer1} \n\n Answer 2: {answer2}"),
#     ]
# )

# Define a new data model for assessing and comparing two answers
class EvaluateAnswers(BaseModel):
    best_answer: str = Field(description="Which answer is more correct, or specify if both are equally valid or invalid.")
    explanation: str = Field(description="Detailed explanation of why one answer is better, both are acceptable, or both are inadequate.")

# Initialize the language model with structured output
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=Openai_API_key)
structured_llm_evaluator = llm.with_structured_output(EvaluateAnswers)

# System prompt to evaluate and compare answers
system = """
Assess the accuracy and relevancy of the two provided answers in relation to the given question. Use the context to determine:
- If each answer directly addresses the question's topic.
- If the answer is factually correct and relevant based on the context.

Note:
- Clearly state if neither answer addresses the question correctly or is factually incorrect.
- Specify which answer, if any, is more relevant and why, even if neither is fully correct.
"""
evaluate_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Context: {context} \n\n Question: {question} \n\n Answer 1: {answer1} \n\n Answer 2: {answer2}"),
    ]
)


# Combine prompt template with structured output
answer_evaluator = evaluate_prompt | structured_llm_evaluator




In [ ]:
Second_answer = input("Please enter your answer: ")

Please enter your answer: Female carriers of the DMD gene, particularly those who show symptoms of the disease, known as manifesting carriers, may experience a range of clinical implications. These may include: 1.	Muscle Weakness: Manifesting carriers may have muscle weakness, particularly in the back, legs, and arms, which can lead to difficulties with mobility and activities that require muscle endurance. 2.	Fatigue: The muscle weakness is often accompanied by an increased susceptibility to fatigue, making it challenging to perform physical tasks that require sustained effort. 3.	Cardiomyopathy: Female carriers are at an increased risk of developing cardiomyopathy, a disease of the heart muscle that can lead to heart failure. Symptoms may include shortness of breath, swelling of the lower extremities, and an inability to do moderate exercise. 4.	Cardiac Issues: Heart problems in manifesting carriers can be serious and potentially life-threatening if left untreated. This underscores t

In [ ]:
evaluation = answer_evaluator.invoke({"context": doc_txt, "question": query_question, "answer1": Answer, "answer2": Second_answer})

print(f"Best Answer: {evaluation.best_answer}")
print(f"Explanation: {evaluation.explanation}")

# Save in document
answer_1 = Answer
answer_2 = Second_answer
question = query_question
best_answer = evaluation.best_answer
explanation = evaluation.explanation

save_best_answer_to_docx(question,answer_1, answer_2, explanation,best_answer )

Best Answer: Answer 2
Explanation: Answer 2 is more relevant and accurate as it directly addresses the clinical implications for female carriers of the DMD gene, particularly those who show symptoms of the disease. It provides a concise and focused list of the potential clinical implications, including muscle weakness, fatigue, cardiomyopathy, reproductive considerations, psychological impact, monitoring, and genetic counseling. Answer 2 also emphasizes the importance of regular monitoring and proper medical care for female carriers exhibiting symptoms.
Document saved as 'Best_Answer_Explanation.docx'


/usr/local/lib/python3.10/dist-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


##Simple RAG

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
from langchain.chat_models import ChatOpenAI
# from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
# LLM
llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.7,  api_key=Openai_API_key)

# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever_text_FAISS,
    chain_type="stuff"
)


In [ ]:
#question = input("Type in your query: \n")
question = Ques_docs[0].page_content
Relevant_docs= retriever_text_FAISS.get_relevant_documents(question)

# for ans in Relevant_docs:
#     print(ans.page_content)

result = qa_chain({"query": question})
result["result"]

'A female dystrophin deficiency carrier may face potential health issues that include weaker muscles in the back, legs, and arms that fatigue easily. They may also experience heart problems, which can show up as shortness of breath or an inability to do moderate exercise. These heart problems, if untreated, can be quite serious, even life-threatening. Some women may also be manifesting carriers and exhibit some signs and symptoms of Duchenne muscular dystrophy (DMD). Regular strength evaluations and close cardiac monitoring are recommended for managing any symptoms that may arise.'

In [ ]:
Relevant_docs

[Document(page_content='For these women, the dystrophin deficiency may result in weaker muscles in the back, legs, and arms that fatigue easily. Manifesting carriers may have heart problems, which can show up as shortness of breath or an inability to do moderate exercise. The heart problems, if untreated, can be quite serious, even life-threatening.\nIn very rare instances, a girl may lack a second X chromosome entirely, or her second X may have sustained serious damage. In these cases, she makes little or no dystrophin (depending on the type of dystrophin mutation), and she develops a dystrophinopathy just as a boy would.\nA female relative of a boy with DMD can get a full range of diagnostic tests to determine her carrier status. If she is found to be a DMD carrier, regular strength evaluations and close cardiac monitoring can help her manage any symptoms that may arise. For more on DMD in females, see Debatable Destinies: Duchenne muscular dystrophy carriers carry on, despite uncert

In [ ]:
result

{'query': '1. What discovery did MDA-supported researchers make in 1986 that advanced our understanding of Duchenne, Becker, and an intermediate form of muscular dystrophies? ',
 'result': 'In 1986, MDA-supported researchers identified a gene on the X chromosome that, when mutated, causes Duchenne, Becker, and an intermediate form of muscular dystrophies. This was a significant advancement in understanding the genetic basis of these conditions.'}

In [ ]:
question

'2. How does a mutation in the gene that encodes the middle section of the dystrophin protein affect the production of the entire protein?'